---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

# Solution 1

## Energy Dataset

### Importing and cleaning the DataFrame 

In [20]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.1f}'.format

In [21]:
import sys
!{sys.executable} -m pip install xlrd

You are using pip version 10.0.1, however version 20.2.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [22]:
# Energy Dataset
# Step 1: Reading Excel in format
energy=pd.read_excel('Energy Indicators.xls',header=[16],usecols='C:G',index_col=0,skipfooter=38) #Skipping header and footer info rows
energy=energy.reset_index().drop(0)  # Dropping unncesary row
energy.columns=['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable'] # Renaming columns

# Step 2: Fitting values
energy.replace('...',np.NaN,inplace=True)
energy['Energy Supply']*=10**6 #Converting to GJ

# Pre-Step3: Cleaning index names
import re # Librería Regex: para analizar strings con caracteres especiales en "Regular Expressions" 
def country_name (str):
    if ( re.search('[\(]',str) is not None ) :       
        return str[:re.search('[\(]',str).start()-1]
    elif (re.search('[\d]',str) is not None):
        return str[:re.search('[\d|]',str).start()]
    else:
        return str

energy['Country']=energy['Country'].apply(country_name)
#energy.set_index('Country',inplace=True)

# Step 3: Renaming indexes
energy=energy.set_index('Country').rename({'Republic of Korea':'South Korea', "United States of America": "United States",
        "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
        "China, Hong Kong Special Administrative Region": "Hong Kong"})
energy

,Energy Supply,Energy Supply per Capita,% Renewable
Country,,,
Afghanistan,"321,000,000.0",10.0,78.7
Albania,"102,000,000.0",35.0,100.0
Algeria,"1,959,000,000.0",51.0,0.6
American Samoa,nan,nan,0.6
Andorra,"9,000,000.0",121.0,88.7
...,...,...,...
Viet Nam,"2,554,000,000.0",28.0,45.3
Wallis and Futuna Islands,0.0,26.0,0.0
Yemen,"344,000,000.0",13.0,0.0


## World Bank

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

In [38]:
GDP=pd.read_csv('world_bank.csv',header=4)
GDP=GDP.rename({'Country Name' : 'Country'})
#GDP=GDP.set_index('Country').rename({"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran","Hong Kong SAR, China": "Hong Kong"})
GDP

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Aruba,ABW,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,"2,467,703,910.6",nan,nan,nan,nan,nan
1,Andorra,AND,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,nan,nan,nan,nan,nan,nan,...,"4,018,195,653.5","4,021,331,428.4","3,675,727,910.5","3,535,389,132.6","3,346,317,328.5","3,185,604,581.5","3,129,537,611.9","3,127,549,907.6",nan,nan
2,Afghanistan,AFG,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,nan,nan,nan,nan,nan,nan,...,"10,305,228,125.1","11,721,187,594.2","12,144,482,858.2","14,697,331,940.6","15,936,800,636.2","16,911,126,453.2","19,352,203,805.6","19,731,337,260.6","19,990,317,160.7","20,294,152,243.0"
3,Angola,AGO,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,nan,nan,nan,nan,nan,nan,...,"55,811,028,114.6","68,420,444,001.5","77,874,196,512.2","79,753,199,398.1","82,470,913,120.7","85,702,615,836.4","90,120,963,241.2","96,261,432,368.3","100,886,286,611.0","103,910,558,934.0"
4,Albania,ALB,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,nan,nan,nan,nan,nan,nan,...,"9,771,760,095.1","10,348,293,941.6","11,127,520,474.3","11,500,292,411.2","11,926,953,258.9","12,231,090,567.7","12,404,772,053.5","12,542,465,022.4","12,793,314,322.8","13,120,823,169.5"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,"Yemen, Rep.",YEM,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,nan,nan,nan,nan,nan,nan,...,"26,725,653,162.7","27,617,869,836.7","28,726,558,529.0","29,914,355,173.6","30,906,753,495.2","26,243,421,035.8","26,891,601,653.7","28,009,137,429.7",nan,nan
260,South Africa,ZAF,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,"77,070,810,928.0","80,034,494,636.6","84,978,442,119.7","91,244,381,889.7","98,489,040,998.3","107,250,656,239.0",...,"340,285,160,027.9","358,526,057,738.4","369,966,781,590.0","364,276,362,720.1","375,349,442,837.2","387,407,362,601.2","396,007,124,238.3","404,768,205,399.9","411,036,853,168.6","416,311,671,520.9"
261,"Congo, Dem. Rep.",COD,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,"15,080,239,239.4","13,443,825,765.2","16,294,010,600.1","17,143,578,705.3","16,725,237,057.9","16,892,046,007.9",...,"16,508,944,144.7","17,542,316,525.5","18,634,483,660.3","19,166,509,369.4","20,523,285,374.2","21,932,132,984.4","23,502,004,276.8","25,500,497,990.0","27,787,759,925.4","29,709,613,590.2"
262,Zambia,ZMB,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,"4,592,974,755.6","4,655,502,678.7","4,539,541,577.2","4,688,093,223.0","5,260,699,172.4","6,136,471,743.3",...,"14,405,693,544.0","15,608,919,909.9","16,822,341,081.1","18,373,419,545.1","20,265,552,104.4","21,403,581,063.3","23,024,380,970.4","24,205,950,559.4","25,422,273,004.7","26,241,270,767.8"


## SCIMAGJR

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [24]:
ScimEn=pd.read_excel('scimagojr-3.xlsx').set_index('Country')
ScimEn

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
Country,,,,,,,
China,1,127050,126767,597237,411683,4.7,138
United States,2,96661,94747,792274,265436,8.2,230
Japan,3,30504,30287,223024,61554,7.3,134
United Kingdom,4,20944,20357,206091,37874,9.8,139
Russian Federation,5,18534,18301,34266,12422,1.9,57
...,...,...,...,...,...,...,...
Guyana,187,1,1,0,0,0.0,0
Christmas Island,188,1,1,0,0,0.0,0
Reunion,189,1,1,2,1,2.0,1


## Merging Datasets

In [25]:
merged_df=pd.merge(ScimEn[:15],pd.merge(energy, GDP.iloc[:,-10:], how='inner',left_index=True,right_index=True) , how='inner', left_index=True, right_index=True)
merged_df.index.name='Country'
merged_df

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.7,138,"127,191,000,000.0",93.0,19.8,"3,992,331,462,570.9","4,559,041,383,517.9","4,997,775,145,606.0","5,459,247,267,275.1","6,039,658,508,485.6","6,612,490,294,273.7","7,124,977,970,399.6","7,672,447,738,017.4","8,230,120,617,793.5","8,797,998,940,421.3"
United States,2,96661,94747,792274,265436,8.2,230,"90,838,000,000.0",286.0,11.6,"14,792,303,791,800.0","15,055,395,304,800.0","15,011,490,541,400.0","14,594,842,181,900.0","14,964,372,000,000.0","15,204,019,634,600.0","15,542,161,722,300.0","15,773,666,098,800.0","16,156,618,476,000.0","16,548,573,278,300.0"
Japan,3,30504,30287,223024,61554,7.3,134,"18,984,000,000.0",149.0,10.2,"5,496,541,517,363.3","5,617,035,943,770.7","5,558,526,873,313.8","5,251,308,399,909.3","5,498,717,815,809.8","5,473,738,114,222.0","5,569,101,602,926.2","5,644,658,523,313.4","5,642,884,448,582.2","5,669,563,143,141.0"
United Kingdom,4,20944,20357,206091,37874,9.8,139,"7,920,000,000.0",124.0,10.6,"2,419,630,700,401.7","2,482,203,235,475.9","2,470,614,405,284.3","2,367,047,611,712.0","2,403,504,326,328.8","2,450,911,011,124.9","2,479,808,627,472.2","2,533,370,110,012.4","2,605,643,189,894.9","2,666,333,396,477.1"
Russian Federation,5,18534,18301,34266,12422,1.9,57,"30,709,000,000.0",214.0,17.3,"1,385,792,607,182.6","1,504,071,117,741.3","1,583,004,071,092.1","1,459,199,142,720.3","1,524,917,468,442.0","1,589,942,641,764.3","1,645,875,899,596.3","1,666,934,123,163.2","1,678,708,855,070.9","1,616,148,858,033.8"
Canada,6,17899,17620,215003,40930,12.0,149,"10,431,000,000.0",296.0,61.9,"1,564,468,853,329.4","1,596,739,898,466.3","1,612,713,061,250.2","1,565,144,676,082.3","1,613,406,134,731.1","1,664,086,754,804.9","1,693,132,927,878.1","1,730,687,770,724.1","1,773,485,821,199.8","1,792,608,764,608.8"
Germany,7,17027,16831,140566,27426,8.3,126,"13,261,000,000.0",165.0,17.9,"3,332,890,754,966.9","3,441,560,834,437.1","3,478,809,377,483.4","3,283,339,933,774.8","3,417,298,013,245.0","3,542,371,125,827.8","3,556,723,774,834.4","3,567,317,390,728.5","3,624,386,278,145.7","3,685,555,907,284.8"
India,8,15005,14841,128763,37209,8.6,115,"33,195,000,000.0",26.0,15.0,"1,265,894,322,131.1","1,374,865,195,244.5","1,428,360,609,657.8","1,549,482,541,544.7","1,708,458,876,829.9","1,821,872,415,623.6","1,924,235,460,945.2","2,051,981,849,791.7","2,200,616,575,311.0","2,367,206,118,959.1"
France,9,13153,12973,130632,28601,9.9,114,"10,597,000,000.0",166.0,17.0,"2,607,839,735,099.3","2,669,423,841,059.6","2,674,637,086,092.7","2,595,966,887,417.2","2,646,994,701,986.8","2,702,031,788,079.5","2,706,968,211,920.5","2,722,566,887,417.2","2,729,631,788,079.5","2,761,185,430,463.6"


In [26]:
def answer_one():
    return merged_df.copy()
answer_one()

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply,Energy Supply per Capita,% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.7,138,"127,191,000,000.0",93.0,19.8,"3,992,331,462,570.9","4,559,041,383,517.9","4,997,775,145,606.0","5,459,247,267,275.1","6,039,658,508,485.6","6,612,490,294,273.7","7,124,977,970,399.6","7,672,447,738,017.4","8,230,120,617,793.5","8,797,998,940,421.3"
United States,2,96661,94747,792274,265436,8.2,230,"90,838,000,000.0",286.0,11.6,"14,792,303,791,800.0","15,055,395,304,800.0","15,011,490,541,400.0","14,594,842,181,900.0","14,964,372,000,000.0","15,204,019,634,600.0","15,542,161,722,300.0","15,773,666,098,800.0","16,156,618,476,000.0","16,548,573,278,300.0"
Japan,3,30504,30287,223024,61554,7.3,134,"18,984,000,000.0",149.0,10.2,"5,496,541,517,363.3","5,617,035,943,770.7","5,558,526,873,313.8","5,251,308,399,909.3","5,498,717,815,809.8","5,473,738,114,222.0","5,569,101,602,926.2","5,644,658,523,313.4","5,642,884,448,582.2","5,669,563,143,141.0"
United Kingdom,4,20944,20357,206091,37874,9.8,139,"7,920,000,000.0",124.0,10.6,"2,419,630,700,401.7","2,482,203,235,475.9","2,470,614,405,284.3","2,367,047,611,712.0","2,403,504,326,328.8","2,450,911,011,124.9","2,479,808,627,472.2","2,533,370,110,012.4","2,605,643,189,894.9","2,666,333,396,477.1"
Russian Federation,5,18534,18301,34266,12422,1.9,57,"30,709,000,000.0",214.0,17.3,"1,385,792,607,182.6","1,504,071,117,741.3","1,583,004,071,092.1","1,459,199,142,720.3","1,524,917,468,442.0","1,589,942,641,764.3","1,645,875,899,596.3","1,666,934,123,163.2","1,678,708,855,070.9","1,616,148,858,033.8"
Canada,6,17899,17620,215003,40930,12.0,149,"10,431,000,000.0",296.0,61.9,"1,564,468,853,329.4","1,596,739,898,466.3","1,612,713,061,250.2","1,565,144,676,082.3","1,613,406,134,731.1","1,664,086,754,804.9","1,693,132,927,878.1","1,730,687,770,724.1","1,773,485,821,199.8","1,792,608,764,608.8"
Germany,7,17027,16831,140566,27426,8.3,126,"13,261,000,000.0",165.0,17.9,"3,332,890,754,966.9","3,441,560,834,437.1","3,478,809,377,483.4","3,283,339,933,774.8","3,417,298,013,245.0","3,542,371,125,827.8","3,556,723,774,834.4","3,567,317,390,728.5","3,624,386,278,145.7","3,685,555,907,284.8"
India,8,15005,14841,128763,37209,8.6,115,"33,195,000,000.0",26.0,15.0,"1,265,894,322,131.1","1,374,865,195,244.5","1,428,360,609,657.8","1,549,482,541,544.7","1,708,458,876,829.9","1,821,872,415,623.6","1,924,235,460,945.2","2,051,981,849,791.7","2,200,616,575,311.0","2,367,206,118,959.1"
France,9,13153,12973,130632,28601,9.9,114,"10,597,000,000.0",166.0,17.0,"2,607,839,735,099.3","2,669,423,841,059.6","2,674,637,086,092.7","2,595,966,887,417.2","2,646,994,701,986.8","2,702,031,788,079.5","2,706,968,211,920.5","2,722,566,887,417.2","2,729,631,788,079.5","2,761,185,430,463.6"


### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [27]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [28]:
def answer_two():
    shape_outer= pd.merge( pd.merge(energy, GDP.iloc[:,-10:], how='outer',left_index=True,right_index=True) , ScimEn[:15], how='outer', left_index=True, right_index=True).shape[0]
    shape_inner=merged_df.shape[0]
    return shape_outer-shape_inner

ValueError: Length mismatch: Expected axis has 264 elements, new values have 1 elements

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [11]:
def answer_three():
    Top15=answer_one()
    Top15['avgGDP']=Top15.iloc[:,-10:].groupby(level=0).apply(np.nanmean,axis=1)
    return Top15['avgGDP'].sort_values(ascending=False)

In [12]:
answer_three()

Country
United States           [15364344302990.0]
China                    [6348608932836.1]
Japan                  [5542207638235.176]
Germany               [3493025339072.8477]
France                 [2681724635761.589]
United Kingdom        [2487906661418.4175]
Brazil                 [2189794143774.905]
Italy                 [2120175089933.7761]
India                   [1769297396603.86]
Canada                 [1660647466307.512]
Russian Federation     [1565459478480.661]
Spain                  [1418078278145.694]
Australia              [1164042729991.427]
South Korea            [1106714508244.852]
Iran                    [444155754051.095]
Name: avgGDP, dtype: object

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [13]:
def answer_four():
    from decimal import Decimal
    gdp6=answer_three().index[5]
    Top15=answer_one()
    return '%.2e' % Decimal(Top15.loc[gdp6,'2015']-Top15.loc[gdp6,'2006'])

In [14]:
answer_four()

'2.47e+11'

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [15]:
def answer_five():
    Top15 = answer_one()
    return Top15['Energy Supply per Capita'].mean()

In [16]:
answer_five()

157.6

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [17]:
def answer_six():
Top15 = answer_one()
return (Top15['% Renewable'].idxmax(),Top15['% Renewable'].max())

IndentationError: expected an indented block (<ipython-input-17-bf1dc1ca1f05>, line 2)

In [ ]:
answer_six()

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [ ]:
def answer_seven():
    Top15 = answer_one()
    Top15['Citation ratio']= Top15['Self-citations']/Top15['Citations']
    return (Top15['Citation ratio'].idxmax(),Top15['Citation ratio'].max())

In [ ]:
answer_seven()

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [ ]:
def answer_eight():
    Top15 = answer_one()
    Top15['Population']= Top15['Energy Supply']/Top15['Energy Supply per Capita']
    return Top15['Population'].nlargest(3).index[-1]

In [ ]:
answer_eight()

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [ ]:
def answer_nine():
    Top15 = answer_one()
    Top15['Citations per Capita']=Top15['Citations'] / (Top15['Energy Supply']/Top15['Energy Supply per Capita'])
    return Top15['Citations per Capita'].corr(Top15['Energy Supply per Capita'])

In [ ]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [ ]:
#plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [ ]:
def answer_ten():
    Top15 = answer_one().set_index('Rank',append=True).sort_index(level=1)
    mean_renew=Top15['% Renewable'].mean()
    Top15['HighRenew']= Top15['% Renewable'].map(lambda r: 1 if r>=mean_renew else 0) 
    return Top15['HighRenew'].reset_index('Rank',drop=True)

In [ ]:
answer_ten()

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [ ]:
def answer_eleven():
    Top15 = answer_one()
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    Top15['EstPop']=Top15['Energy Supply']/Top15['Energy Supply per Capita']
    Top15['Continent']=pd.Series(ContinentDict)
    return Top15[['EstPop','Continent']].groupby('Continent').agg(['size','sum','mean','std'])

In [ ]:
answer_eleven()

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [ ]:
def answer_twelve():
    Top15 = answer_one()
    
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
    Top15['Continent']=pd.Series(ContinentDict)
    Top15['% Renewable']=pd.cut(Top15['% Renewable'],5)#.groupby('Continent')                                       
    bins=Top15.groupby(['Continent','% Renewable']).size()
    return bins[bins>0]

In [ ]:
answer_twelve()

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [ ]:
def answer_thirteen():
    Top15 = answer_one()
    Top15['PopEst']=(Top15['Energy Supply']/Top15['Energy Supply per Capita'])
    return Top15['PopEst'].map(lambda p: str( '{:,}'.format(p) ) )

In [ ]:
#np.get_printoptions()
answer_thirteen()

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [ ]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [ ]:
plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!